In [1]:
import pandas as pd
import numpy as np
from map_functions import *

df = pd.read_csv('data/locations (no_lat-lng).csv')
df.tail()

,Name_Orig,Lat_Orig,Lng_Orig,Name_Des,Lat_Des,Lng_Des
21,"Columbus, OH",NaN,NaN,"Dublin, Ireland",NaN,NaN
22,"Columbus, OH",NaN,NaN,"Fort Worth, TX",NaN,NaN
23,"Columbus, OH",NaN,NaN,"Wooster, OH",NaN,NaN
24,"Columbus, OH",NaN,NaN,"Gambier, OH",NaN,NaN
25,"Columbus, OH",NaN,NaN,"Norwalk, CT",NaN,NaN


In [2]:
# Add columns for origin and destination payloads (for Google API) 
payload_pre = 'https://maps.googleapis.com/maps/api/geocode/json?address='
df['Payload_Orig'] = df['Name_Orig'].str.replace(', ','+').str.replace(' ','+')
df['Payload_Des'] = df['Name_Des'].str.replace(', ','+').str.replace(' ','+')
df['Payload_Orig'] = payload_pre + df['Payload_Orig']
df['Payload_Des'] = payload_pre + df['Payload_Des']
df['Distance (nm)'] = 0
df['Distance (mi)'] = 0
df['Distance (km)'] = 0
df.tail()

,Name_Orig,Lat_Orig,Lng_Orig,Name_Des,Lat_Des,Lng_Des,Payload_Orig,Payload_Des,Distance (nm),Distance (mi),Distance (km)
21,"Columbus, OH",NaN,NaN,"Dublin, Ireland",NaN,NaN,https://maps.googleapis.com/maps/api/geocode/j...,https://maps.googleapis.com/maps/api/geocode/j...,0,0,0
22,"Columbus, OH",NaN,NaN,"Fort Worth, TX",NaN,NaN,https://maps.googleapis.com/maps/api/geocode/j...,https://maps.googleapis.com/maps/api/geocode/j...,0,0,0
23,"Columbus, OH",NaN,NaN,"Wooster, OH",NaN,NaN,https://maps.googleapis.com/maps/api/geocode/j...,https://maps.googleapis.com/maps/api/geocode/j...,0,0,0
24,"Columbus, OH",NaN,NaN,"Gambier, OH",NaN,NaN,https://maps.googleapis.com/maps/api/geocode/j...,https://maps.googleapis.com/maps/api/geocode/j...,0,0,0
25,"Columbus, OH",NaN,NaN,"Norwalk, CT",NaN,NaN,https://maps.googleapis.com/maps/api/geocode/j...,https://maps.googleapis.com/maps/api/geocode/j...,0,0,0


I round the decimal degrees and distance measurements for readability. Pick what to round decimal degrees to based on the accuracy required. For our purposes, 3 decimal places should suffice. 

This table [from Wikipedia](https://en.wikipedia.org/wiki/Decimal_degrees#Precision) gives the precision of decimal degrees. 

| Decimal Degrees	| Identifiable	| Distance At Equator | 
|------|------|------|
|1	| country or large region	| 111.32 km |
|0.1	| large city or district	| 11.132 km |
|0.01	| town or village	| 1.1132 km |
|0.001	| neighborhood, street	| 111.32 m |
|0.0001	| individual street, land parcel	| 11.132 m |
|0.00001	| individual trees	| 1.1132 m |
|0.000001	| individual humans	| 111.32 mm |
|0.0000001	| practical limit of commercial surveying	| 11.132 mm |
|0.00000001	| specialized surveying (e.g. tectonic plate mapping)	| 1.1132 mm |

In [3]:
# Find latitude and longitude of each location, and distance between
round_coord = 3
round_dist = 1

print('Completed...')
for row in range(0,df.shape[0]):
    try: 
        # Read payloads to pass to Google API
        payload_orig = df.iloc[row,6]
        payload_des = df.iloc[row,7]
        
        # Request and write latitude and longitude
        lat_orig, lng_orig = get_lat_lng(payload_orig)
        lat_des, lng_des = get_lat_lng(payload_des)
        df.ix[row,'Lat_Orig'] = round(lat_orig,round_coord)
        df.ix[row,'Lng_Orig'] = round(lng_orig,round_coord)
        df.ix[row,'Lat_Des'] = round(lat_des,round_coord)
        df.ix[row,'Lng_Des'] = round(lng_des,round_coord)
        
        # Calculate and write distance
        nm, mi, km = get_distance(lat_orig, lng_orig, lat_des, lng_des)
        df.ix[row,'Distance (nm)']  = round(nm,round_dist)
        df.ix[row,'Distance (mi)']  = round(mi,round_dist)
        df.ix[row,'Distance (km)'] = round(km,round_dist)
        
        print(' - '+ str(round(nm,round_dist))+' nm '+df.iloc[row,0]+' to '+df.iloc[row,3])
    
    except:
        print('\tSKIPPING: '+df.iloc[row,0]+' to '+df.iloc[row,3])
        print('\t'+payload_orig)
        print('\t'+payload_des)

df.tail()

Completed...
 - 157.2 nm Torrance, CA to Paso Robles, CA
 - 1547.4 nm Torrance, CA to Mobile, AL
 - 21.3 nm Torrance, CA to Anaheim, CA
 - 322.9 nm Torrance, CA to Sacramento, CA
 - 96.9 nm Sacramento, CA to Reno, NV
 - 29.5 nm Torrance, CA to Avalon, CA
 - 90.2 nm Torrance, CA to Coronado, CA
 - 159.2 nm Torrance, CA to Sequoia National Park
 - 94.9 nm Sequoia National Park to Yosemite National Park
 - 100.6 nm Yosemite National Park to Reno, NV
 - 31.9 nm Torrance, CA to Thousand Oaks, CA
 - 332.6 nm Torrance, CA to Zion National Park
 - 405.5 nm Zion National Park to Denver, CO
 - 392.5 nm Denver, CO to Manhattan, KS
 - 300.3 nm Manhattan, KS to St. Louis, MO
 - 227.9 nm St. Louis, MO to Chicago, IL
 - 63.0 nm Chicago, IL to South Bend, IN
 - 180.4 nm South Bend, IN to Columbus, OH
 - 559.4 nm Columbus, OH to Boston, MA
 - 52.6 nm Boston, MA to Wellfleet, MA
 - 180.4 nm Columbus, OH to South Bend, IN
 - 3110.3 nm Columbus, OH to Dublin, Ireland
 - 816.3 nm Columbus, OH to Fort Worth

,Name_Orig,Lat_Orig,Lng_Orig,Name_Des,Lat_Des,Lng_Des,Payload_Orig,Payload_Des,Distance (nm),Distance (mi),Distance (km)
21,"Columbus, OH",39.961,-82.999,"Dublin, Ireland",53.350,-6.260,https://maps.googleapis.com/maps/api/geocode/j...,https://maps.googleapis.com/maps/api/geocode/j...,3110.3,3579.3,5760.3
22,"Columbus, OH",39.961,-82.999,"Fort Worth, TX",32.755,-97.331,https://maps.googleapis.com/maps/api/geocode/j...,https://maps.googleapis.com/maps/api/geocode/j...,816.3,939.3,1511.7
23,"Columbus, OH",39.961,-82.999,"Wooster, OH",40.805,-81.935,https://maps.googleapis.com/maps/api/geocode/j...,https://maps.googleapis.com/maps/api/geocode/j...,70.3,80.9,130.1
24,"Columbus, OH",39.961,-82.999,"Gambier, OH",40.376,-82.397,https://maps.googleapis.com/maps/api/geocode/j...,https://maps.googleapis.com/maps/api/geocode/j...,37.2,42.8,68.9
25,"Columbus, OH",39.961,-82.999,"Norwalk, CT",41.118,-73.408,https://maps.googleapis.com/maps/api/geocode/j...,https://maps.googleapis.com/maps/api/geocode/j...,443.9,510.9,822.1


In [4]:
# Save df to csv
header = ['Name_Orig','Lat_Orig','Lng_Orig','Name_Des','Lat_Des','Lng_Des',
          'Distance (nm)','Distance (mi)','Distance (km)']
df.to_csv('data/locations.csv', columns=header, index=False)